In [1]:
# libraries needed 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
import re
%matplotlib inline

# libraries we might not need
import csv

### Using the following code
to use the whole document you only need one file specified by filepath for the time being

In [2]:
%%time
# imports a random sample of size s from csv-file as a pandas dataframe
# pandas using python 3.X uses utf-8 encoding

# usage: specify file location, sample size and seed(used by random)
filepath = '/home/daniel/Downloads/clean-100k.csv'
#filepath = 'news_sample.csv' # <- overwrite for setup
s = 250                      # desired sample size(seems to have slack ie. not exact)
seed = 1                     # seed used by Pseudorandom number generator

# init dataframe with specified values
df = pd.read_csv(filepath).sample(n=s, random_state=seed)

# set dataframe index to article id - use df.to_csv('tmp.csv', index=True, header=True) as index=True writes index to csv with index_name
df.set_index('id', inplace=True)

# visual output
print(df.shape, '<- size of dataframe \n')
df.head()

(250, 16) <- size of dataframe 

CPU times: user 2.31 s, sys: 187 ms, total: 2.5 s
Wall time: 2.5 s


,Unnamed: 0,meta_description,updated_at,content,authors,scraped_at,url,meta_keywords,summary,domain,title,keywords,tags,type,inserted_at,source
id,,,,,,,,,,,,,,,,
48770,3660,World View: Both Greece and European Leaders S...,2018-02-02 01:19:41.756664,this morning s key headlines from generational...,John J. Xenakis,2018-01-25 20:13:50.426130,http://www.breitbart.com/national-security/201...,"['Alexis Tsipras', 'Angela Merkel', 'Britain',...",NaN,breitbart.com,World View: Both Greece and European Leaders S...,NaN,"Tunisia, Generational Dynamics, Britain, Alexi...",political,2018-02-02 01:19:41.756632,NaN
96003,7278,NaN,2018-02-02 01:19:41.756664,daily app fix april <number> th <number> risk ...,NaN,2018-01-25 20:13:50.426130,http://beforeitsnews.com/science-and-technolog...,[''],NaN,beforeitsnews.com,"Daily App Fix April 20th 2013 – RISK, Connect ...",NaN,NaN,fake,2018-02-02 01:19:41.756632,NaN
16182,4317,NaN,2018-02-02 01:19:41.756664,consciousness and unified physics are the keys...,Waking Times,2018-01-25 16:17:44.789555,http://beforeitsnews.com/alternative/2016/07/c...,[''],NaN,beforeitsnews.com,Consciousness and Unified Physics are the Keys...,NaN,NaN,fake,2018-02-02 01:19:41.756632,NaN
89922,1932,NaN,2018-02-02 01:19:41.756664,"do something , anything headline : bitcoin blo...",Humble Student Of The Markets,2018-01-25 20:13:50.426130,http://beforeitsnews.com/financial-markets/201...,[''],NaN,beforeitsnews.com,"Do something, anything!",NaN,NaN,fake,2018-02-02 01:19:41.756632,NaN
106668,5321,NaN,2018-02-02 01:19:41.756664,from conservapedia transfer pricing is the set...,NaN,2018-01-25 20:13:50.426130,http://www.conservapedia.com/Transfer_Pricing,[''],NaN,conservapedia.com,Transfer Pricing,NaN,NaN,bias,2018-02-02 01:19:41.756632,NaN


### data-tables: [name]-uniq / relational-tables: [name]_in
creating csv-files for database

In [3]:
# specify where to save all csv-files
path = 'Database_CSV_IN/'

# create temporary dataframe and use article id as index 
out_df = pd.DataFrame({'id':df.index})
out_df.set_index('id', inplace=True)

In [4]:
### types_uniq - data-table ###
type_array = df.type.unique() # get array of unique types
type_df = pd.DataFrame({'id': np.arange(type_array.size), 'name':type_array})

# write file and free memory
type_df.to_csv(path + 'type_clean.csv', index=False, header=True)
#del type_array
#del type_df # tmp delete later

In [5]:
### tags_uniq - data-table ###

# creates list of list but formaly it is a pd.series of lists
tags_series_of_lists = df.tags.dropna().str.split(', ') # -> ', ' not ','

if not 'tags' in out_df: ### tmp need another method ###
    out_df.insert(0,column = 'tags', value = tags_series_of_lists)

# flattern tags_series_of_lists to a set(ie. unique values only)
tags_list = list(set([item for sublist in tags_series_of_lists for item in sublist]))

# create dataframe
tags_df = pd.DataFrame({'id': np.arange(len(tags_list)), 'name':tags_list})

# write file and free memory
tags_df.to_csv(path + 'tags_clean.csv', index=False, header=True)
del tags_series_of_lists
del tags_list

In [6]:
### tags_in - relational-table ###

# get all pairs of article_id and tags in a article (for all articles)
articles_id_tags_name_pairs_df = out_df.tags.dropna().explode()

# split tags_name and articles_id
articles_id_array = articles_id_tags_name_pairs_df.index.to_numpy()
tags_name_array = articles_id_tags_name_pairs_df.to_numpy()

# create dict with tag_name as key - [swap tags with tags_id]
tags_name_as_key_df = tags_df.set_index('name')
tags_dict = tags_name_as_key_df['id'].to_dict()

# replace tags with tag id
tags_id = np.array([tags_dict[key] for key in tags_name_array])

# create dataframe
tags_in_df = pd.DataFrame(data=articles_id_array, index=tags_id, columns=['article_id'])
tags_in_df.index.name='tags_id'

# write file and free memory
tags_in_df.to_csv(path + 'tags_in.csv', index=True, header=True)

In [7]:
### authors-uniq - data-table ###

# creates list of list but formaly it is a pd.series of lists
authors_series_of_lists = df.authors.dropna().str.split(',') # -> ',' not ', '

if not 'authors' in out_df: ### tmp need another method ###
    out_df.insert(0,column = 'authors', value = authors_series_of_lists)

# flattern authors_series_of_lists to a set(ie. unique values only)
authors_list = list(set([item for sublist in authors_series_of_lists for item in sublist]))

# create dataframe
authors_df = pd.DataFrame({'id': np.arange(len(authors_list)), 'name':authors_list})

# write file and free memory
authors_df.to_csv(path + 'authors_clean.csv', index=False, header=True)
#del authors_series_of_lists
#del authors_list
#del authors_df

In [8]:
### authors_in - relational-table ###

# get all pairs of article_id and authors in a article (for all articles)
articles_id_authors_name_pairs_df = out_df.authors.dropna().explode()

# split authors_name and articles_id
articles_id_array = articles_id_authors_name_pairs_df.index.to_numpy()
authors_name_array = articles_id_authors_name_pairs_df.to_numpy()

# create dict with tag_name as key - [swap authors with authors_id]
authors_name_as_key_df = authors_df.set_index('name')
authors_dict = authors_name_as_key_df['id'].to_dict()

# replace authors with tag id
authors_id = np.array([authors_dict[key] for key in authors_name_array])

# create dataframe
authors_in_df = pd.DataFrame(data=articles_id_array, index=authors_id, columns=['article_id'])
authors_in_df.index.name='authors_id'

# write file and free memory
authors_in_df.to_csv(path + 'authors_in.csv', index=True, header=True)

In [9]:
### domains-uniq - data-table ###
domain_array = df.domain.unique() # get array of unique domains
domain_df = pd.DataFrame({'id': np.arange(domain_array.size), 'name':domain_array})

# write file and free memory
domain_df.to_csv(path + 'domain_name_clean.csv', index=False, header=True)
#del domain_array
#del domain_df

In [10]:
### meta_keywords_uniq - data-table ###

# use regex to remove string-padding
regex = r" *['\"\[\]]+"
meta_keywords_series = df.meta_keywords.replace(to_replace=regex, value='', regex=True).str.split(',')
#meta_keywords_series = meta_keywords_series.replace(r'', np.NaN)

if not 'meta_keywords' in out_df: ### tmp need another method ###
    out_df.insert(0,column = 'meta_keywords', value = meta_keywords_series)

# create array of unique
meta_keywords_set = meta_keywords_series.explode().unique()

# create dataframe
meta_keywords_df = pd.DataFrame({'id': np.arange(len(meta_keywords_set)), 'name':meta_keywords_set})

# write file and free memory
meta_keywords_df.to_csv(path + 'meta_keywords_clean.csv', index=False, header=True)
#del meta_keywords_series
#del meta_keywords_set
#del meta_keywords_list
#del meta_keywords_df

In [11]:
### meta_keywords_in - relational-table ###

# get all pairs of article_id and meta_keywords in a article (for all articles)
articles_id_meta_keywords_name_pairs_df = out_df.meta_keywords.dropna().explode()

# split meta_keywords_name and articles_id
articles_id_array = articles_id_meta_keywords_name_pairs_df.index.to_numpy()
meta_keywords_name_array = articles_id_meta_keywords_name_pairs_df.to_numpy()

# create dict with tag_name as key - [swap meta_keywords with meta_keywords_id]
meta_keywords_name_as_key_df = meta_keywords_df.set_index('name')
meta_keywords_dict = meta_keywords_name_as_key_df['id'].to_dict()

# replace meta_keywords with tag id
meta_keywords_id = np.array([meta_keywords_dict[key] for key in meta_keywords_name_array])

# create dataframe
meta_keywords_in_df = pd.DataFrame(data=articles_id_array, index=meta_keywords_id, columns=['article_id'])
meta_keywords_in_df.index.name='meta_keywords_id'

# write file and free memory
meta_keywords_in_df.to_csv(path + 'meta_keywords_in.csv', index=True, header=True)

### depricated - might be useful later

In [12]:
# row-function used for extracting article and uniq_[name] id
def article_and_uniq_name_id(row, name_df):
    # test for overlap(isin) between article_row and uniq_[name] and create np_array
    tmp_df = name_df[name_df['name'].isin(row[0])] # MAYBE FASTER -> ONLY RETURN ONE COLUMN
    tmp_np_array = tmp_df.values
    
    # create np_array with row_id for each uniq_[name]
    row_id = np.full((np.size(tmp_np_array,0), 1), row.name)
    
    # combine
    tmp_np_array = np.append(tmp_np_array, row_id, axis=1)

    #print('>', tmp_df)
    # change numer of columns to return [col_1, col2, ...]
    return tmp_np_array[:, [0, -1]] # MAYBE NOT NEEDED IF CHAGED TO ONE ROW IE. FASTER

In [13]:
# relational tags_in

#tags_series_of_lists = out_df[['tags']].dropna().apply(article_and_uniq_name_id, axis='columns', args=(tags_df, )) 
tags_series_of_lists = out_df[['tags']].dropna().apply(article_and_uniq_name_id, axis='columns', args=(tags_df, )) 

# concat series into one np_array
tags_in_ndarray = [element for list_ in tags_series_of_lists for element in list_]

# create dataframe
tags_in_df = pd.DataFrame(data = tags_in_ndarray, columns = ['tag_id', 'article_id'])

# write file and free memory
tags_in_df.to_csv(path + 'tags_in.csv', index=True, index_label='id', header=True)

In [14]:
# relational authors_in

# series of np_arrays - 
authors_series_of_lists = out_df[['authors']].dropna().apply(article_and_uniq_name_id, axis='columns', args=(authors_df, )) 

# concat series into one np_array
authors_in_ndarray = [element for list_ in authors_series_of_lists for element in list_]

# create dataframe
authors_in_df = pd.DataFrame(data = authors_in_ndarray, columns = ['tag_id', 'article_id'])

# write file and free memory
authors_in_df.to_csv(path + 'authors_in.csv', index=True, index_label='id', header=True)

In [15]:
authors_in_ndarray = np.array([element for list_ in authors_series_of_lists for element in list_])
authors_in_ndarray

array([[148, 48770],
       [57, 16182],
       [139, 89922],
       [109, 6149],
       [120, 6149],
       [189, 6149],
       [102, 37524],
       [33, 80069],
       [81, 80069],
       [97, 80069],
       [13, 10733],
       [35, 97547],
       [157, 97547],
       [3, 106395],
       [10, 106395],
       [17, 106395],
       [49, 106395],
       [77, 106395],
       [80, 106395],
       [86, 106395],
       [105, 106395],
       [132, 106395],
       [164, 106395],
       [206, 84637],
       [124, 36062],
       [64, 50587],
       [73, 101062],
       [87, 101062],
       [127, 101062],
       [162, 101062],
       [193, 36200],
       [79, 17232],
       [186, 25174],
       [89, 17821],
       [187, 30390],
       [87, 102852],
       [127, 102852],
       [162, 102852],
       [0, 72915],
       [11, 72915],
       [31, 72915],
       [83, 72915],
       [87, 72915],
       [127, 72915],
       [162, 72915],
       [197, 72915],
       [207, 72915],
       [214, 72915],
    

In [16]:
%load_ext line_profiler

In [17]:
def myFunc():
    return
%lprun -f myFunc myFunc()

Timer unit: 1e-06 s

Total time: 1e-06 s
File: <ipython-input-17-a1528040687e>
Function: myFunc at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def myFunc():
     2         1          1.0      1.0    100.0      return